In [1]:
import os
import torch as t
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.multiprocessing import Process
import transformers
import json
from io import StringIO

import sys
sys.path.append('/home/ubuntu/mlab/days')
sys.path.append('/home/paperspace/mlab/days')
import utils

DEVICES=[0,1]



def import_lesswrong_corpus(filename = "lw_corpus.json"):
    with open(filename) as file:
        return json.load(file)
    return None

In [2]:
corpus = import_lesswrong_corpus('small_lw_corpus.json')

In [3]:
def corpus_to_tensor_list(corpus, max_tokens=1024):
    tokenizer = transformers.GPT2TokenizerFast.from_pretrained("gpt2")
    
    tensor_list = []
    current_tokens, staged_tokens = [], []
    
    for i, d in enumerate(corpus):
        print("Time for the: ", i)
        while True:
            if not len(staged_tokens) >= max_tokens:
                staged_tokens = tokenizer(d['title'] + d['text'] + '<|endoftext|>').input_ids
            while len(current_tokens) < max_tokens and len(staged_tokens) > max_tokens:

                space_in_current = max_tokens - len(current_tokens)            
                current_tokens += staged_tokens[0:space_in_current]
                staged_tokens = staged_tokens[space_in_current:None]
                print(len(staged_tokens), ":", staged_tokens)

            tensor_list.append(t.tensor(current_tokens))
            current_tokens = []
            break
        
    tensor_list.append(t.tensor(current_tokens))
    return tensor_list

In [4]:
import transformers
tokenizer = transformers.GPT2TokenizerFast.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [5]:
corpus = import_lesswrong_corpus()
t_list = corpus_to_tensor_list(corpus, 10)
for tensor in t_list:
    tokens = tokenizer.convert_ids_to_tokens(tensor)
    print([token.replace('Ġ', '') for token in tokens])

Token indices sequence length is longer than the specified maximum sequence length for this model (2966 > 1024). Running this sequence through the model will result in indexing errors


Time for the:  0
572 : [290, 1692, 4430, 79, 220, 220, 1875, 220, 220, 309, 220, 220, 289, 220, 220, 1312, 220, 220, 264, 220, 220, 220, 220, 220, 220, 1312, 220, 220, 264, 220, 220, 220, 220, 220, 220, 257, 220, 220, 220, 220, 220, 220, 256, 220, 220, 374, 220, 220, 257, 220, 220, 299, 220, 220, 264, 220, 220, 269, 220, 220, 374, 220, 220, 1312, 220, 220, 279, 220, 220, 256, 220, 220, 220, 220, 220, 220, 267, 220, 220, 277, 220, 220, 220, 220, 220, 220, 257, 220, 220, 220, 220, 220, 220, 269, 220, 220, 267, 220, 220, 299, 220, 220, 410, 220, 220, 304, 220, 220, 374, 220, 220, 264, 220, 220, 257, 220, 220, 256, 220, 220, 1312, 220, 220, 267, 220, 220, 299, 220, 220, 220, 220, 220, 220, 275, 220, 220, 304, 220, 220, 256, 220, 220, 266, 220, 220, 304, 220, 220, 304, 220, 220, 299, 220, 220, 220, 220, 220, 220, 350, 220, 220, 257, 220, 220, 334, 220, 220, 300, 220, 220, 220, 220, 220, 220, 327, 220, 220, 289, 220, 220, 374, 220, 220, 1312, 220, 220, 264, 220, 220, 256, 220, 220, 1312, 220

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



['ads', 'to', 'the', 'models', 'on', 'average', 'learning', 'to', 'defect', 'faster']
['than', 'with', 'no', 'outer', 'loop', '.', 'This', 'is', 'presumably', 'because']
['of', 'the', 'additional', 'optim', 'ising', 'power', 'applied', 'to', 'the', 'task']
['of', 'my', 'opic', 'reward', 'maxim', 'isation', '.', 'We', 'also', 'see']
['that', 'the', 'average', 'cooperation', 'probability', 'does', 'not', 'reach', 'values', 'near']
['1', 'for', 'anything', '(', 'in', 'contrast', 'to', 'short', '-', 'loop']
['P', 'BT', ',', 'as', 'seen', 'above', ').', 'The', 'paper', 'does']
['not', 'say', 'anything', 'about', 'whether', 'this', 'is', 'because', 'some', 'fraction']
['learn', 'to', 'cooperate', 'repeatedly', 'and', 'the', 'rest', 'to', 'defect', 'repeatedly']
[',', 'or', 'whether', 'all', 'agents', 'cooperate', 'at', 'roughly', 'the', 'same']
['probability', '.', 'It', 'seems', 'more', 'likely', 'to', 'be', 'the', 'former']
['.', 'Note', 'that', 'the', 'policy', 'of', 'the', 'agent', 'is',